In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold

In [64]:
df = pd.read_csv('./data/veg_model_data.csv')
df.head()

,vegetable,flavor_family,texture,prepared
0,artichokes,mild/neutral,soft-chewy,roast
1,artichokes,mild/neutral,soft-chewy,steam
2,arugula,spicy,soft-chewy,salad
3,arugula,spicy,soft-chewy,salad
4,asparagus,grassy,crunchy,salad


In [65]:
df_dummy = pd.get_dummies(df, columns=df.columns[1:], drop_first=True)

In [66]:
X = df.drop(columns='vegetable')
y = df['vegetable']

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [68]:
# rfc = RandomForestClassifier()

In [69]:
# knn = KNeighborsClassifier()

In [ ]:
# cross_val_score(knn, X_train, y_train).mean()

In [71]:
import category_encoders as ce

In [74]:
encoder = ce.OneHotEncoder(cols=df.columns[1:])
df_enc = encoder.fit_transform(df)

C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [75]:
df_enc

,vegetable,flavor_family_1,flavor_family_2,flavor_family_3,flavor_family_4,flavor_family_5,texture_1,texture_2,texture_3,texture_4,texture_5,prepared_1,prepared_2,prepared_3,prepared_4,prepared_5,prepared_6,prepared_7,prepared_8
0,artichokes,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1,artichokes,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,arugula,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,arugula,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,asparagus,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,turnips,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
84,garlic,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
85,garlic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
86,lettuce,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [81]:
X = df_enc.drop(columns='vegetable').astype(float)
y = df_enc['vegetable']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [82]:
tf.random.set_seed(42)

In [83]:
n_input = X_train.shape[1]

In [86]:
model = Sequential()
model.add(Dense(15, input_shape=(n_input,), activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [87]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, callbacks=[early_stop])

Epoch 1/20


UnimplementedError:  Cast string to float is not supported
	 [[node categorical_crossentropy/Cast (defined at <ipython-input-87-a716fff89f9e>:1) ]] [Op:__inference_train_function_2954]

Function call stack:
train_function


In [97]:
# encoder = LabelEncoder()
# encoder.fit(Y)
# encoded_Y = encoder.transform(Y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)
 
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\keras\wrappers\scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\keras\wrappers\scikit_learn.py", line 162, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\keras\engine\training.py", line 1184, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\tensorflow\python\eager\def_f

Baseline: nan% (nan%)


C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\keras\wrappers\scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\keras\wrappers\scikit_learn.py", line 162, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\keras\engine\training.py", line 1184, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\Adam\anaconda3\envs\dsi\lib\site-packages\tensorflow\python\eager\def_f